In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

# 기본 데이터 확인

In [2]:
df_2026 = pd.read_csv("./tmdb_v3_output/tmdb_2026.csv")
df_2026.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
0,Avengers: Doomsday,2026-12-18,movie,NaN,NaN,NaN,Science Fiction,"Joe Russo, Anthony Russo","Chris Hemsworth, Vanessa Kirby, Anthony Mackie...",US,en,"Marvel Studios, Kevin Feige Productions, AGBO","The Avengers, Wakandans, Fantastic Four, Thund...","superhero, based on comic, sequel, superhero t...",0.0,0
1,28 Years Later Part 2: The Bone Temple,2026-01-06,movie,NaN,NaN,NaN,"Horror, Science Fiction, Thriller",Nia DaCosta,"Aaron Taylor-Johnson, Jack O'Connell, Emma Lai...",US,en,"DNA Films, Decibel Films, Columbia Pictures, BFI",The second installment in the 28 Days Later se...,NaN,0.0,0
2,Shrek 5,2026-12-23,movie,NaN,NaN,NaN,"Animation, Family, Comedy, Fantasy","Walt Dohrn, Conrad Vernon","Mike Myers, Eddie Murphy, Cameron Diaz, Zendaya",US,en,DreamWorks Animation,The fifth film in the Shrek franchise. Plot TBA.,"fairy tale, sequel, ogre, animation",0.0,0
3,Toy Story 5,2026-06-17,movie,NaN,NaN,NaN,"Animation, Family, Comedy",Andrew Stanton,"Tom Hanks, Tim Allen, Joan Cusack, Ernie Hudso...",US,en,Pixar,"Buzz, Woody, Jessie and the rest of the gang's...","sequel, toy comes to life",0.0,0
4,Spider-Man: Brand New Day,2026-07-30,movie,NaN,NaN,NaN,"Science Fiction, Action, Adventure",Destin Daniel Cretton,"Tom Holland, Sadie Sink, Liza Colón-Zayas",US,en,"Marvel Studios, Columbia Pictures, Pascal Pict...",The fourth installment in the MCU Spider-Man f...,"superhero, based on comic, marvel cinematic un...",0.0,0


In [3]:
df_2026.sort_values(by='title', ascending=True).head(10)

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
1,28 Years Later Part 2: The Bone Temple,2026-01-06,movie,NaN,NaN,NaN,"Horror, Science Fiction, Thriller",Nia DaCosta,"Aaron Taylor-Johnson, Jack O'Connell, Emma Lai...",US,en,"DNA Films, Decibel Films, Columbia Pictures, BFI",The second installment in the 28 Days Later se...,NaN,0.0,0
268,3xHelô,2026-12-31,movie,NaN,NaN,NaN,Drama,Lucas Pelaes,"Laura Paro, Lia Diwonko, Valéria Lima",BR,pt,"rena, produtora de arte",A girl just broke up with her boyfriend and ne...,"romance, teenage romance, teen drama, failed r...",0.0,0
257,5D,2026-01-01,movie,12.0,NaN,NaN,"Drama, Thriller",Lance Sorenson,"Chloe Rodriques, Jacob Coggshall, Zofia Ossowska",US,en,NaN,A grieving woman reflects on the toxic relatio...,NaN,0.0,0
367,A Blank Frame,2026-01-27,movie,NaN,NaN,NaN,"Drama, Thriller",Kanan Aliyev,"Araz Nabibeyli, Kanan Aliyev",AZ,en,NaN,A filmmaker learns that the process of making ...,NaN,0.0,0
195,A Gift Wrapped in Time,2026-12-25,movie,NaN,NaN,NaN,"Romance, TV Movie",Jacob Chaffee,NaN,US,en,Chaffee Pictures,A workaholic gets a second chance at love when...,"lost love, christmas spirit",0.0,0
172,A Guardian's Heart,2026-05-15,movie,NaN,NaN,NaN,"Action, Adventure, Thriller",R. Michael Givens,"Alexis Baca, Laura Mason, Wayne D Burdette Jr.",US,en,"Heart Stopper Productions, Moonless Media",Jake Bishop has been an exemplary Executive Pr...,independent film,0.0,0
239,A Hill to Die On,2026-05-04,movie,NaN,NaN,NaN,Comedy,Peter Blair,NaN,US,en,NaN,This dark comedy follows Jamie Van Hamm as he ...,NaN,0.0,0
221,A Palette for Murder,2026-09-04,movie,NaN,NaN,NaN,"Mystery, Comedy, Crime, Thriller",Joseph Shedd,"Michael Goyet, Kody Myers, Jacob Taylor, Josep...",US,en,Dead by Dawn Studios,Six seeming unrelated men receive mysterious i...,NaN,0.0,0
210,A Sinful People,2026-05-04,movie,NaN,NaN,NaN,Thriller,Andrew Lamm,NaN,US,en,NaN,A reformed heroin addict turned Christian man ...,NaN,0.0,0
226,A Sinful People,2026-05-04,movie,NaN,NaN,NaN,Thriller,Andrew Lamm,NaN,US,en,NaN,A reformed heroin addict turned Christian man ...,NaN,0.0,0


In [4]:
df_2026.isnull().sum()

title                   0
release_date            0
type                    0
runtime               311
season_count          376
episode_count         376
genre                  78
director               62
cast                  107
country                 6
language                0
production_company    122
synopsis               87
tags                  261
imdb_rating             0
imdb_review_count       0
dtype: int64

# 2. 전처리 전처리_type

In [5]:
df_2026[df_2026['season_count'].isna() & df_2026['episode_count'].isna()].head(5)

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
0,Avengers: Doomsday,2026-12-18,movie,NaN,NaN,NaN,Science Fiction,"Joe Russo, Anthony Russo","Chris Hemsworth, Vanessa Kirby, Anthony Mackie...",US,en,"Marvel Studios, Kevin Feige Productions, AGBO","The Avengers, Wakandans, Fantastic Four, Thund...","superhero, based on comic, sequel, superhero t...",0.0,0
1,28 Years Later Part 2: The Bone Temple,2026-01-06,movie,NaN,NaN,NaN,"Horror, Science Fiction, Thriller",Nia DaCosta,"Aaron Taylor-Johnson, Jack O'Connell, Emma Lai...",US,en,"DNA Films, Decibel Films, Columbia Pictures, BFI",The second installment in the 28 Days Later se...,NaN,0.0,0
2,Shrek 5,2026-12-23,movie,NaN,NaN,NaN,"Animation, Family, Comedy, Fantasy","Walt Dohrn, Conrad Vernon","Mike Myers, Eddie Murphy, Cameron Diaz, Zendaya",US,en,DreamWorks Animation,The fifth film in the Shrek franchise. Plot TBA.,"fairy tale, sequel, ogre, animation",0.0,0
3,Toy Story 5,2026-06-17,movie,NaN,NaN,NaN,"Animation, Family, Comedy",Andrew Stanton,"Tom Hanks, Tim Allen, Joan Cusack, Ernie Hudso...",US,en,Pixar,"Buzz, Woody, Jessie and the rest of the gang's...","sequel, toy comes to life",0.0,0
4,Spider-Man: Brand New Day,2026-07-30,movie,NaN,NaN,NaN,"Science Fiction, Action, Adventure",Destin Daniel Cretton,"Tom Holland, Sadie Sink, Liza Colón-Zayas",US,en,"Marvel Studios, Columbia Pictures, Pascal Pict...",The fourth installment in the MCU Spider-Man f...,"superhero, based on comic, marvel cinematic un...",0.0,0


In [6]:
df_2026[df_2026['season_count'].isna() & df_2026['episode_count'].isna()].shape

(376, 16)

In [7]:
df_2026['type'].value_counts(dropna=False)

type
movie    376
tv        32
Name: count, dtype: int64

In [8]:
print(df_2026[df_2026['type'] == 'movie']['season_count'].value_counts(dropna=False))
print(df_2026[df_2026['type'] == 'movie']['episode_count'].value_counts(dropna=False))

season_count
NaN    376
Name: count, dtype: int64
episode_count
NaN    376
Name: count, dtype: int64


In [9]:
print(df_2026[df_2026['type'] == 'tv']['season_count'].value_counts(dropna=False))

season_count
1.0    32
Name: count, dtype: int64


In [10]:
print(df_2026[df_2026['type'] == 'tv']['episode_count'].value_counts(dropna=False))

episode_count
1.0     28
4.0      1
8.0      1
15.0     1
6.0      1
Name: count, dtype: int64


In [11]:
df_2026_1 = df_2026

In [12]:
# movie인 경우 season/episode count가 NaN이면 1로 채움
df_2026_1.loc[df_2026_1['type'] == 'movie', ['season_count', 'episode_count']] = df_2026_1.loc[df_2026_1['type'] == 'movie', ['season_count', 'episode_count']].fillna(1)
pd.set_option('display.max_columns', None)
df_2026_1.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
0,Avengers: Doomsday,2026-12-18,movie,NaN,1.0,1.0,Science Fiction,"Joe Russo, Anthony Russo","Chris Hemsworth, Vanessa Kirby, Anthony Mackie...",US,en,"Marvel Studios, Kevin Feige Productions, AGBO","The Avengers, Wakandans, Fantastic Four, Thund...","superhero, based on comic, sequel, superhero t...",0.0,0
1,28 Years Later Part 2: The Bone Temple,2026-01-06,movie,NaN,1.0,1.0,"Horror, Science Fiction, Thriller",Nia DaCosta,"Aaron Taylor-Johnson, Jack O'Connell, Emma Lai...",US,en,"DNA Films, Decibel Films, Columbia Pictures, BFI",The second installment in the 28 Days Later se...,NaN,0.0,0
2,Shrek 5,2026-12-23,movie,NaN,1.0,1.0,"Animation, Family, Comedy, Fantasy","Walt Dohrn, Conrad Vernon","Mike Myers, Eddie Murphy, Cameron Diaz, Zendaya",US,en,DreamWorks Animation,The fifth film in the Shrek franchise. Plot TBA.,"fairy tale, sequel, ogre, animation",0.0,0
3,Toy Story 5,2026-06-17,movie,NaN,1.0,1.0,"Animation, Family, Comedy",Andrew Stanton,"Tom Hanks, Tim Allen, Joan Cusack, Ernie Hudso...",US,en,Pixar,"Buzz, Woody, Jessie and the rest of the gang's...","sequel, toy comes to life",0.0,0
4,Spider-Man: Brand New Day,2026-07-30,movie,NaN,1.0,1.0,"Science Fiction, Action, Adventure",Destin Daniel Cretton,"Tom Holland, Sadie Sink, Liza Colón-Zayas",US,en,"Marvel Studios, Columbia Pictures, Pascal Pict...",The fourth installment in the MCU Spider-Man f...,"superhero, based on comic, marvel cinematic un...",0.0,0


In [13]:
df_2026[(df_2026['type'] == 'tv') & (df_2026['season_count'].isna() | df_2026['episode_count'].isna())]
#df_2026[(df_2026['type'] == 'movie') & (df_2026['season_count'].isna() | df_2026['episode_count'].isna())]

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count


In [14]:
print(df_2026_1[df_2026_1['season_count'] == 1.0]['episode_count'].value_counts(dropna=False))

episode_count
1.0     404
4.0       1
8.0       1
15.0      1
6.0       1
Name: count, dtype: int64


In [15]:
print(df_2026_1[df_2026_1['episode_count'] == 1.0]['season_count'].value_counts(dropna=False))

season_count
1.0    404
Name: count, dtype: int64


In [16]:
df_2026_2 = df_2026_1
df_2026_2.loc[df_2026_2['type'] == 'tv', ['season_count', 'episode_count']] = df_2026_2.loc[df_2026_1['type'] == 'tv', ['season_count', 'episode_count']].fillna(1)
pd.set_option('display.max_columns', None)
df_2026_2.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
0,Avengers: Doomsday,2026-12-18,movie,NaN,1.0,1.0,Science Fiction,"Joe Russo, Anthony Russo","Chris Hemsworth, Vanessa Kirby, Anthony Mackie...",US,en,"Marvel Studios, Kevin Feige Productions, AGBO","The Avengers, Wakandans, Fantastic Four, Thund...","superhero, based on comic, sequel, superhero t...",0.0,0
1,28 Years Later Part 2: The Bone Temple,2026-01-06,movie,NaN,1.0,1.0,"Horror, Science Fiction, Thriller",Nia DaCosta,"Aaron Taylor-Johnson, Jack O'Connell, Emma Lai...",US,en,"DNA Films, Decibel Films, Columbia Pictures, BFI",The second installment in the 28 Days Later se...,NaN,0.0,0
2,Shrek 5,2026-12-23,movie,NaN,1.0,1.0,"Animation, Family, Comedy, Fantasy","Walt Dohrn, Conrad Vernon","Mike Myers, Eddie Murphy, Cameron Diaz, Zendaya",US,en,DreamWorks Animation,The fifth film in the Shrek franchise. Plot TBA.,"fairy tale, sequel, ogre, animation",0.0,0
3,Toy Story 5,2026-06-17,movie,NaN,1.0,1.0,"Animation, Family, Comedy",Andrew Stanton,"Tom Hanks, Tim Allen, Joan Cusack, Ernie Hudso...",US,en,Pixar,"Buzz, Woody, Jessie and the rest of the gang's...","sequel, toy comes to life",0.0,0
4,Spider-Man: Brand New Day,2026-07-30,movie,NaN,1.0,1.0,"Science Fiction, Action, Adventure",Destin Daniel Cretton,"Tom Holland, Sadie Sink, Liza Colón-Zayas",US,en,"Marvel Studios, Columbia Pictures, Pascal Pict...",The fourth installment in the MCU Spider-Man f...,"superhero, based on comic, marvel cinematic un...",0.0,0


# 2. 데이터 전처리_runtime

In [17]:
df_2026_2.isnull().sum()

title                   0
release_date            0
type                    0
runtime               311
season_count            0
episode_count           0
genre                  78
director               62
cast                  107
country                 6
language                0
production_company    122
synopsis               87
tags                  261
imdb_rating             0
imdb_review_count       0
dtype: int64

In [18]:
df_2026_2.shape

(408, 16)

In [19]:
df_2026_2[df_2026_2['runtime'].isna() & df_2026_2['genre'].isna() ].shape

(65, 16)

## 2_2 runtime null 값 열 제거

In [20]:
df_2026_3 = df_2026_2

In [21]:
df_2026_3.dropna(subset=['runtime'], inplace=True)

In [22]:
df_2026_3.shape

(97, 16)

# 2. 데이터 전처리_pr

In [23]:
df_2026_4 = df_2026_3
df_2026_4

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
15,Return to Silent Hill,2026-01-22,movie,96.0,1.0,1.0,"Horror, Mystery",Christophe Gans,"Jeremy Irvine, Hannah Emily Anderson, Evie Tem...",NaN,en,"Konami, Davis Films, Hassell Free Productions,...",When a mysterious letter calls James back to S...,based on video game,0.0,0
28,The Rip,2026-01-15,movie,133.0,1.0,1.0,"Crime, Thriller",Joe Carnahan,"Matt Damon, Ben Affleck, Teyana Taylor, Sasha ...",US,en,Artists Equity,After a team of Miami cops discovers millions ...,"miami, florida",0.0,0
48,Urchin,2026-02-11,movie,99.0,1.0,1.0,"Comedy, Drama",Harris Dickinson,"Frank Dillane, Megan Northam, Amr Waked, Karyn...",GB,en,"Devisio Pictures, BBC Film, Tricky Knot, Somesuch","Mike, a rough sleeper in London, trapped in a ...","london, england",0.0,0
50,Смешарики: Сквозь вселенные,2026-08-13,movie,85.0,1.0,1.0,"Science Fiction, Adventure","Andrey Marmontov, Ilya Maksimov","Nikolai Dobrynin, Yan Tsapnik, Ivan Agapov, Ly...",RU,ru,Riki Group,Smeshariki Krosh and Yozhik find an unusual de...,NaN,0.0,0
82,BORN SEXY YESTERDAY: Decoding the Enigma of Be...,2026-01-30,movie,60.0,1.0,1.0,"Documentary, Crime, Drama, Mystery, History",NaN,"Emunah La-Paz, Lavender Lashay, Aleks Montijah...",US,en,Little Ant Productions LLC.,"October 1, 1985. The chilling emptiness in Mic...","women's sexual identity, women's suffrage, soc...",0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,The cycle,2026-02-27,movie,90.0,1.0,1.0,NaN,Tyler Knowles,"Tyler Knowles, Nightenday, pixelatedart3495, G...",US,en,NaN,In the near future a new drug outlawed by the ...,NaN,0.0,0
376,Désenchantées,2026-01-01,tv,52.0,1.0,4.0,"Drama, Mystery, Crime",David Hourrègue,"Constance Labbé, Marie Denarnaud, Elodie Frenc...",FR,fr,"Banijay Studios France, France Télévisions, Pi...",NaN,NaN,0.0,0
378,Você Me Vira a Cabeça,2026-01-01,tv,45.0,1.0,8.0,"Comedy, Drama",NaN,NaN,BR,pt,Charles Morais - Produção Cultural,After a video of her singing goes viral on the...,NaN,0.0,0
386,Funnyville,2026-04-22,tv,15.0,1.0,1.0,"Comedy, Animation, Action & Adventure",NaN,"Yaakov Mijares, Ulysses Gontes, Logan Taylor",US,en,Mijares-Gontes Productions,"Three adolescents, Victor, Eric, and Jake livi...",NaN,0.0,0


In [24]:
df_2026_4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97 entries, 15 to 387
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               97 non-null     object 
 1   release_date        97 non-null     object 
 2   type                97 non-null     object 
 3   runtime             97 non-null     float64
 4   season_count        97 non-null     float64
 5   episode_count       97 non-null     float64
 6   genre               84 non-null     object 
 7   director            88 non-null     object 
 8   cast                80 non-null     object 
 9   country             95 non-null     object 
 10  language            97 non-null     object 
 11  production_company  66 non-null     object 
 12  synopsis            94 non-null     object 
 13  tags                44 non-null     object 
 14  imdb_rating         97 non-null     float64
 15  imdb_review_count   97 non-null     int64  
dtypes: float64(4)

### tags 관련 기본 문제 해결

1) synopsis, tags, genre가 모두 NaN인 데이터 모두 제거

In [25]:
# 1. 세기: synopsis, genre가 모두 NaN인 행 개수
nan_rows = df_2026_4[['synopsis', 'genre']].isna().all(axis=1)
count_nan_rows = nan_rows.sum()
print(f"synopsis, genre가 모두 NaN인 행 개수: {count_nan_rows}")

synopsis, genre가 모두 NaN인 행 개수: 0


In [26]:
# 2. 삭제: 그런 행 삭제
df_2026_4_cleaned = df_2026_4.loc[~nan_rows].copy()

In [27]:
df_2026_4 = df_2026_4.loc[~nan_rows].copy()
df_2026_4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97 entries, 15 to 387
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               97 non-null     object 
 1   release_date        97 non-null     object 
 2   type                97 non-null     object 
 3   runtime             97 non-null     float64
 4   season_count        97 non-null     float64
 5   episode_count       97 non-null     float64
 6   genre               84 non-null     object 
 7   director            88 non-null     object 
 8   cast                80 non-null     object 
 9   country             95 non-null     object 
 10  language            97 non-null     object 
 11  production_company  66 non-null     object 
 12  synopsis            94 non-null     object 
 13  tags                44 non-null     object 
 14  imdb_rating         97 non-null     float64
 15  imdb_review_count   97 non-null     int64  
dtypes: float64(4)

In [28]:
df_2026_4

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count
15,Return to Silent Hill,2026-01-22,movie,96.0,1.0,1.0,"Horror, Mystery",Christophe Gans,"Jeremy Irvine, Hannah Emily Anderson, Evie Tem...",NaN,en,"Konami, Davis Films, Hassell Free Productions,...",When a mysterious letter calls James back to S...,based on video game,0.0,0
28,The Rip,2026-01-15,movie,133.0,1.0,1.0,"Crime, Thriller",Joe Carnahan,"Matt Damon, Ben Affleck, Teyana Taylor, Sasha ...",US,en,Artists Equity,After a team of Miami cops discovers millions ...,"miami, florida",0.0,0
48,Urchin,2026-02-11,movie,99.0,1.0,1.0,"Comedy, Drama",Harris Dickinson,"Frank Dillane, Megan Northam, Amr Waked, Karyn...",GB,en,"Devisio Pictures, BBC Film, Tricky Knot, Somesuch","Mike, a rough sleeper in London, trapped in a ...","london, england",0.0,0
50,Смешарики: Сквозь вселенные,2026-08-13,movie,85.0,1.0,1.0,"Science Fiction, Adventure","Andrey Marmontov, Ilya Maksimov","Nikolai Dobrynin, Yan Tsapnik, Ivan Agapov, Ly...",RU,ru,Riki Group,Smeshariki Krosh and Yozhik find an unusual de...,NaN,0.0,0
82,BORN SEXY YESTERDAY: Decoding the Enigma of Be...,2026-01-30,movie,60.0,1.0,1.0,"Documentary, Crime, Drama, Mystery, History",NaN,"Emunah La-Paz, Lavender Lashay, Aleks Montijah...",US,en,Little Ant Productions LLC.,"October 1, 1985. The chilling emptiness in Mic...","women's sexual identity, women's suffrage, soc...",0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,The cycle,2026-02-27,movie,90.0,1.0,1.0,NaN,Tyler Knowles,"Tyler Knowles, Nightenday, pixelatedart3495, G...",US,en,NaN,In the near future a new drug outlawed by the ...,NaN,0.0,0
376,Désenchantées,2026-01-01,tv,52.0,1.0,4.0,"Drama, Mystery, Crime",David Hourrègue,"Constance Labbé, Marie Denarnaud, Elodie Frenc...",FR,fr,"Banijay Studios France, France Télévisions, Pi...",NaN,NaN,0.0,0
378,Você Me Vira a Cabeça,2026-01-01,tv,45.0,1.0,8.0,"Comedy, Drama",NaN,NaN,BR,pt,Charles Morais - Produção Cultural,After a video of her singing goes viral on the...,NaN,0.0,0
386,Funnyville,2026-04-22,tv,15.0,1.0,1.0,"Comedy, Animation, Action & Adventure",NaN,"Yaakov Mijares, Ulysses Gontes, Logan Taylor",US,en,Mijares-Gontes Productions,"Three adolescents, Victor, Eric, and Jake livi...",NaN,0.0,0


## director와 cast가 모두 nan인 행 제거

In [29]:
nan_both = df_2026_4[(df_2026_4['director'].isna()) & (df_2026_4['cast'].isna())]
print(f"director와 cast 둘 다 NaN인 행 개수: {len(nan_both)}")

director와 cast 둘 다 NaN인 행 개수: 6


In [30]:
df_2026_4 = df_2026_4.dropna(subset=['director', 'cast'], how='all')

In [31]:
df_2026_4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91 entries, 15 to 387
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               91 non-null     object 
 1   release_date        91 non-null     object 
 2   type                91 non-null     object 
 3   runtime             91 non-null     float64
 4   season_count        91 non-null     float64
 5   episode_count       91 non-null     float64
 6   genre               80 non-null     object 
 7   director            88 non-null     object 
 8   cast                80 non-null     object 
 9   country             89 non-null     object 
 10  language            91 non-null     object 
 11  production_company  65 non-null     object 
 12  synopsis            90 non-null     object 
 13  tags                43 non-null     object 
 14  imdb_rating         91 non-null     float64
 15  imdb_review_count   91 non-null     int64  
dtypes: float64(4)

# tag npl처리

In [32]:
# 1. tags 결측치는 빈 문자열로 대체
df_2026_4['tags'] = df_2026_4['tags'].fillna('')

# 2. synopsis, genre 결측치 처리 후 text_input 생성
df_2026_4['synopsis'] = df_2026_4['synopsis'].fillna('')
df_2026_4['genre'] = df_2026_4['genre'].fillna('')
df_2026_4['text_input'] = df_2026_4['synopsis'] + ' ' + df_2026_4['genre']

# 3. tags 기준으로 train / test 나누기
train_df = df_2026_4[df_2026_4['tags'] != ''].copy()
test_df = df_2026_4[df_2026_4['tags'] == ''].copy()

# 4. tags 리스트로 분리
train_df['tags_list'] = train_df['tags'].apply(lambda x: x.split(','))

# 5. 다중 라벨 인코딩
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(train_df['tags_list'])

# 6. 텍스트 벡터화 (TF-IDF)
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_df['text_input'])

# 7. 모델 학습 (LogisticRegression 다중 라벨)
model = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

# 8. 예측 대상(test_df)이 비어있지 않으면 예측 수행
if not test_df.empty:
    X_test = vectorizer.transform(test_df['text_input'])
    y_prob = model.predict_proba(X_test)

    # 예측 임계값 설정 (낮춰서 태그 생성 늘림)
    threshold = 0.07
    y_pred = (y_prob >= threshold).astype(int)

    predicted_tags = mlb.inverse_transform(y_pred)
    test_df['predicted_tags'] = [', '.join(tags) if tags else np.nan for tags in predicted_tags]

    # 9. 규칙 기반 태그 보완용 키워드 룰 정의
    rules = {
        'romance': ['romance', 'love', 'relationship', 'affair', 'heart', 'kiss', 'wedding',
                    'boyfriend', 'girlfriend', 'breakup', 'dating', 'first love',
                    'valentine', 'romantic', 'jealousy', 'intimacy', 'honeymoon'],
        'action': ['battle', 'war', 'fight', 'soldier', 'gun', 'explosion', 'chase', 'enemy',
                'car chase', 'explosives', 'gunfight', 'hand-to-hand', 'sniper',
                'ambush', 'mercenary', 'adrenaline', 'combat', 'high-octane'],
        'thriller': ['thriller', 'mystery', 'suspense', 'crime', 'killer', 'murder', 'detective',
                    'interrogation', 'conspiracy', 'mind game', 'tension', 'abduction',
                    'dangerous', 'mind-bending', 'ticking clock', 'trap'],
        'comedy': ['comedy', 'funny', 'humor', 'laugh', 'joke', 'parody',
                'sitcom', 'sarcasm', 'one-liner', 'slapstick', 'banter',
                'awkward moment', 'gag', 'prank', 'hilarity'],
        'horror': ['ghost', 'horror', 'zombie', 'demon', 'vampire', 'curse',
                'haunted', 'scream', 'paranormal', 'supernatural', 'bloodcurdling',
                'creepy', 'terrifying', 'evil spirit', 'possessed', 'ritual'],
        'fantasy': ['magic', 'wizard', 'dragon', 'kingdom', 'mythical',
                    'sorcerer', 'enchanted', 'portal', 'spell', 'epic quest',
                    'mystical creature', 'ancient prophecy', 'grimoire'],
        'sci-fi': ['space', 'alien', 'robot', 'future', 'technology', 'sci-fi', 'cyber',
                'cyborg', 'AI uprising', 'time machine', 'spaceship', 'hyperspace',
                'galaxy', 'terraforming', 'futuristic', 'parallel universe', 'bioengineering'],
        'drama': ['family', 'life', 'struggle', 'emotional', 'drama', 'society',
                'inner struggle', 'self-discovery', 'character driven', 'emotional arc',
                'breakdown', 'life lesson', 'interpersonal', 'catharsis'],
        'crime': ['criminal', 'robbery', 'heist', 'mafia', 'gang',
                'heist', 'detective', 'undercover', 'criminal syndicate',
                'drug cartel', 'interrogation room', 'forensics', 'manhunt'],
        'animation': ['animation', 'cartoon', 'drawn', 'animated'],
        'sports': ['sports', 'football', 'basketball', 'coach', 'tournament'],
        'historical': ['king', 'queen', 'dynasty', 'empire', 'history', 'ancient'],
        'adventure': ['journey', 'quest', 'treasure', 'explore', 'island', 'survival'],
        'musical': ['song', 'singing', 'performance', 'dance', 'stage', 'music'],
        'documentary': ['real', 'true story', 'interview', 'archive', 'biography', 'footage'],
        'psychological': ['mind', 'mental', 'insanity', 'delusion', 'subconscious', 'dream'],
        'superhero': ['hero', 'superpower', 'villain', 'save', 'city', 'mask'],
        'noir': ['detective', 'dark', 'cigarette', 'femme fatale', 'shadow', 'monologue'],
        'family': ['parent', 'child', 'home', 'holiday', 'siblings', 'bond'],
        'rom-com': ['love', 'funny', 'awkward', 'date', 'charm', 'happy ending'],
        'military': ['army', 'navy', 'mission', 'rank', 'base', 'operation'],
        'cyberpunk': ['neon', 'hacker', 'corporation', 'dystopia', 'implant', 'surveillance'],
        'post-apocalyptic': ['ruins', 'survivor', 'radiation', 'collapse', 'tribe', 'scavenge'],
        'teen': ['high school', 'prom', 'friendship', 'coming of age', 'crush', 'rebellion'],
        'feel-good': ['inspiring', 'heartwarming', 'uplifting', 'positive', 'hope', 'smile'],
        'dark': ['depressing', 'grim', 'tragic', 'bleak', 'morbid'],
        'female-lead': ['strong woman', 'heroine', 'female protagonist', 'girl power'],
        'twist': ['plot twist', 'unpredictable', 'reveal', 'surprise ending'],
        'based-on-true-story': ['true story', 'biopic', 'real events', 'true events'],
        'underdog': ['struggle', 'rise', 'overcome', 'unlikely hero', 'against all odds'],
        'ensemble': ['many characters', 'multiple leads', 'group dynamic', 'interwoven stories'],
        'nostalgic': ['retro', 'classic', 'throwback', '80s', '90s', 'childhood'],
        'social-issue': ['racism', 'inequality', 'poverty', 'justice', 'activism'],
        'trending-theme': ['AI', 'climate', 'pandemic', 'influencer', 'metaverse'],
        'revenge': ['revenge', 'vengeance', 'payback', 'retaliation', 'score to settle'],
        'detective': ['detective', 'investigation', 'clues', 'case', 'inspector', 'sleuth'],
        'time-travel': ['time travel', 'time loop', 'past and future', 'timeline', 'chronosphere'],
        'courtroom': ['courtroom', 'trial', 'lawyer', 'judge', 'defendant', 'testimony'],
        'survival': ['survival', 'wilderness', 'isolation', 'extreme conditions', 'barely escape'],
        'espionage': ['spy', 'espionage', 'agent', 'undercover', 'intel', 'classified'],
        'campus': ['college', 'campus', 'lecture hall', 'seniors', 'dorm', 'professor'],
        'startup': ['startup', 'entrepreneur', 'founder', 'venture', 'pitching', 'angel investor'],
        'heist': ['heist', 'robbery', 'master plan', 'getaway', 'vault', 'crew'],
        'royalty': ['prince', 'princess', 'royal', 'crown', 'throne', 'palace'],
        'wuxia': ['martial arts', 'swordplay', 'wuxia', 'clan', 'sect', 'inner energy'],
        'idol': ['idol', 'stage', 'fandom', 'trainee', 'debut', 'fan meeting'],
        'political': ['election', 'politics', 'scandal', 'congress', 'corruption', 'regime'],
        'revenge-romance': ['bitter love', 'masked identity', 'burning passion', 'payback with love'],
        'healing-romance': ['gentle romance', 'emotional support', 'quiet relationship', 'warm affection'],
        'coming-of-age': ['self-discovery', 'growing up', 'teenage years', 'puberty',
                        'youth', 'maturity', 'identity', 'high school', 'first time', 'life lessons'],
        'dark-comedy': ['dark humor', 'macabre laugh', 'twisted joke', 'morbidly funny',
                        'ironic death', 'satirical tragedy', 'deadpan', 'black comedy'],
        'dystopian': ['oppressive regime', 'totalitarian', 'no freedom', 'surveillance state',
                    'rebellion', 'social collapse', 'harsh world', 'authoritarian', 'controlled society'],
        'slice-of-life': ['daily routine', 'ordinary moments', 'casual life', 'everyday drama',
                        'subtle emotion', 'mundane beauty', 'life as it is', 'quiet narrative'],
        'found-family': ['bond', 'unrelated family', 'loyal companions', 'chosen family',
                        'support system', 'teamwork', 'squad', 'surrogate parents', 'emotional ties'],
        'road-trip': ['on the road', 'journey across', 'miles', 'car travel', 'hit the road',
                    'cross-country', 'travel companions', 'gas station', 'stopover', 'long drive'],
        'tech-noir': ['grim future', 'tech-savvy criminal', 'neon-lit', 'cyber corruption',
                    'urban decay', 'AI rebellion', 'cyberspace', 'digital heist', 'surveillance overload'],
        'folklore': ['legend', 'traditional tale', 'mythical creature', 'ancient lore',
                    'cultural story', 'spirit', 'oral tradition', 'forest demon', 'village tale']
    }

    def simple_tag_rule(text):
        text = text.lower()
        tags = set()
        for tag, keywords in rules.items():
            if any(word in text for word in keywords):
                tags.add(tag)
        return ', '.join(tags) if tags else np.nan

    # 10. 예측 실패 (nan)인 경우 규칙 기반으로 태그 보완
    no_tag_idx = test_df[test_df['predicted_tags'].isna()].index
    test_df.loc[no_tag_idx, 'predicted_tags'] = test_df.loc[no_tag_idx, 'text_input'].apply(simple_tag_rule)

    # 11. 원본에 태그 삽입
    df_2026_4.loc[test_df.index, 'tags'] = test_df['predicted_tags']

C:\Users\User\AppData\Local\Temp\ipykernel_33760\1529960442.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2026_4['tags'] = df_2026_4['tags'].fillna('')
C:\Users\User\AppData\Local\Temp\ipykernel_33760\1529960442.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2026_4['synopsis'] = df_2026_4['synopsis'].fillna('')
C:\Users\User\AppData\Local\Temp\ipykernel_33760\1529960442.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [33]:
df_2026_4['tags'].value_counts(dropna=False)

tags
 family,  short film, dark comedy                                                                                                                                                                                                                                                                                                                                                                                                                      32
 family,  romance,  short film, dark comedy                                                                                                                                                                                                                                                                                                                                                                                                             8
 family,  short film, dark comedy, epic                                                                        

# country에서 nan은 language와 같은 값으로 처리

In [34]:
df_2026_4.isnull().sum()

title                  0
release_date           0
type                   0
runtime                0
season_count           0
episode_count          0
genre                  0
director               3
cast                  11
country                2
language               0
production_company    26
synopsis               0
tags                   0
imdb_rating            0
imdb_review_count      0
text_input             0
dtype: int64

In [35]:
df_2026_4['country'].fillna(df_2026_4['language'], inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_33760\3297427359.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_2026_4['country'].fillna(df_2026_4['language'], inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_33760\3297427359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2026_4['country'].fillna(df_2026_4['language'], inpla

In [36]:
df_2026_4.isnull().sum()

title                  0
release_date           0
type                   0
runtime                0
season_count           0
episode_count          0
genre                  0
director               3
cast                  11
country                0
language               0
production_company    26
synopsis               0
tags                   0
imdb_rating            0
imdb_review_count      0
text_input             0
dtype: int64

In [37]:
df_2026_4['production_company'].value_counts(dropna=False)

production_company
NaN                                                                                                                                                      26
Josiah productions                                                                                                                                        4
Walt Nick's Pictures, MAVV, A. J. Luna Producciones                                                                                                       4
LYCU Studios                                                                                                                                              3
Prism Productions                                                                                                                                         2
Bougie Arabs Entertainment, We Make Movies, LiLifornia Diaries Productions                                                                                2
Superstructure, Auvergne-Rhône-Alpes Cinéma, 

# production_company, director, cast _is_comming 처리
- 결측 값이 많아 이렇게 진행
- 정보 부족이지 하나의 회사명, 이름은 x
- 'unknown'만 삽입했을 경우 정상 값이라고 학습하는 것을 방지 => unknown으로 채우되 값은 없음으로 지정하기 위해 사용

In [38]:
unique_count = df_2026_4['production_company'].nunique(dropna=True)
print(unique_count)

51


In [39]:
df_2026_4['production_company_is_missing'] = df_2026_4['production_company'].isna().astype(int)
df_2026_4['production_company'] = df_2026_4['production_company'].fillna('unknown')

C:\Users\User\AppData\Local\Temp\ipykernel_33760\3742739270.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2026_4['production_company_is_missing'] = df_2026_4['production_company'].isna().astype(int)
C:\Users\User\AppData\Local\Temp\ipykernel_33760\3742739270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2026_4['production_company'] = df_2026_4['production_company'].fillna('unknown')


In [40]:
df_2026_4['cast_is_missing'] = df_2026_4['cast'].isna().astype(int)
df_2026_4['cast'] = df_2026_4['cast'].fillna('unknown')

C:\Users\User\AppData\Local\Temp\ipykernel_33760\2162327797.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2026_4['cast_is_missing'] = df_2026_4['cast'].isna().astype(int)
C:\Users\User\AppData\Local\Temp\ipykernel_33760\2162327797.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2026_4['cast'] = df_2026_4['cast'].fillna('unknown')


In [41]:
df_2026_4['director_is_missing'] = df_2026_4['director'].isna().astype(int)
df_2026_4['director'] = df_2026_4['director'].fillna('unknown')

C:\Users\User\AppData\Local\Temp\ipykernel_33760\2973637657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2026_4['director_is_missing'] = df_2026_4['director'].isna().astype(int)
C:\Users\User\AppData\Local\Temp\ipykernel_33760\2973637657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2026_4['director'] = df_2026_4['director'].fillna('unknown')


In [42]:
print(df_2026_4.shape)
print(df_2026_4.isnull().sum())

(91, 20)
title                            0
release_date                     0
type                             0
runtime                          0
season_count                     0
episode_count                    0
genre                            0
director                         0
cast                             0
country                          0
language                         0
production_company               0
synopsis                         0
tags                             0
imdb_rating                      0
imdb_review_count                0
text_input                       0
production_company_is_missing    0
cast_is_missing                  0
director_is_missing              0
dtype: int64


In [43]:
nan_tags_rows = df_2026_4[df_2026_4['tags'].isna()]
nan_tags_rows

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,cast_is_missing,director_is_missing


In [44]:
df_2026_5 = df_2026_4[df_2026_4['tags'].notna()].copy()
df_2026_5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91 entries, 15 to 387
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   title                          91 non-null     object 
 1   release_date                   91 non-null     object 
 2   type                           91 non-null     object 
 3   runtime                        91 non-null     float64
 4   season_count                   91 non-null     float64
 5   episode_count                  91 non-null     float64
 6   genre                          91 non-null     object 
 7   director                       91 non-null     object 
 8   cast                           91 non-null     object 
 9   country                        91 non-null     object 
 10  language                       91 non-null     object 
 11  production_company             91 non-null     object 
 12  synopsis                       91 non-null     object 


In [45]:
df_2026_5.to_csv('전처리_완료(csv)/2026년도 전처리.csv', index=False)